# Tutorial for 3D object detection
This tutorial is about 3D object detection based on Lidar point cloud. The project is about implementing a newly proposed Triple Attention module to enhance the performance of PointPillar.

First downlaod [MobaXterm](https://mobaxterm.mobatek.net/download.html) for Windows users.(This is necessary if you want to foward the demo display that originally shown in GPU Farm to your local computer)

For ubuntu users, just use the terminal provided by the OS itself and connect to GPU farm with -X option, e.g. ssh -X 

For Mac users, install XQuartz.
```shell
brew cask install xquartz
```

If you are using gpu farm phase 1 which has pre-installed cuda-10.2, you can skip the installation of CUDA. 

If you are using gpu farm phase 2, you have to install CUDA 10.2 manually.

### Manually install CUDA 10.2 (skip this if you are using gpu farm phase 2)
```shell
gpu-interactive
cd ~
wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda_10.2.89_440.33.01_linux.run
chmod 777 cuda_10.2.89_440.33.01_linux.run
./cuda_10.2.89_440.33.01_linux.run --toolkit --toolkitpath=$HOME/cuda-10.2 --defaultroot=$HOME/cuda-10.2
```

## Before excuting this jupyter notebook file, first open a terminal to prepare the environment and download the KITTI dataset.
## 1. Installation of OpenPCDet environment
### Create conda environment
(skip the command gpu-interactive if you have already executed before)
```shell
gpu-interactive
conda create -n openpcdet python=3.7
conda activate openpcdet
```

### Install required packages (pytorch, mayavi, etc.)
```shell
conda install pytorch==1.7.0 torchvision==0.8.0 -c pytorch
conda install mayavi -c conda-forge
pip install scipy==1.7.3
```

### Install spconv pre-compiled for cuda-10.2
```shell
pip install spconv-cu102
```

### Install OpenPCDet
```shell
cd
git clone https://github.com/tianqi-wang1996/OpenPCDet.git
cd ~/OpenPCDet
```
if you are using gpu farm phase 1:
```shell
CUDA_HOME=/usr/local/cuda-10.2 python setup.py develop
```
if you are using gpu farm phase 2 which has manually installed CUDA-10.2 in the previous steps:
```shell
CUDA_HOME=$HOME/cuda-10.2 python setup.py develop
```
### Install Jupyter notebook
```shell
conda install jupyter notebook
```

## 2. Download KITTI dataset, unzip and put the data under OpenPCDet/data/kitti/
### Download and unzip KITTI dataset into ~/OpenPCDet/data/kitti folder
```shell
cd ~/OpenPCDet/data/kitti 
wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_calib.zip
wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_velodyne.zip
unzip data_object_calib.zip
unzip data_object_image_2.zip
unzip data_object_velodyne.zip
```
### Install gdown to download files from Google Drive links
```shell
pip install gdown
```
### Download the road_plane folder and unzip it to data/kitti/training/ folder
```shell
gdown --id 1d5mq0RXRnvHPVeKx6Q612z0YRO1t2wAp
unzip train_planes.zip -d ~/OpenPCDet/data/kitti/training
```
### Generate the info files that needed for training
```shell
cd ~/OpenPCDet
python -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos tools/cfgs/dataset_configs/kitti_dataset.yaml
```

## 3. Visualize the detection result
### Download a pre-trained checkpoint file for PointPillar and use the demo.py to check the visualization result
```shell
cd ~/OpenPCDet
mkdir pretrained_ckpt
cd ~/OpenPCDet/pretrained_ckpt
gdown --id 1wMxWTpU1qUoY3DsCH31WJmvJxcjFXKlm
```
#### Important: 
1. You have to launch this jupyter notebook file (.ipynb) or start your local terminal from MobaXterm for Windows or XQuartz for Mac or regular terminal for Ubuntu.
2. make sure you have activated openpcdet environment in your terminal before launching the jupyter notebook, or executing further commands in your terminal.
3. If you choose to launch jupyter notebook, choose the kernel which shown in Kernel/Change kernel to Python 3 (ipykernel)

#### From the next jupyter cell, you can just simply excute the following cells in jupyter notebook.
#### If you prefer execute in your local terminal, then exclude the "%" or "!" symbols ahead of each line, e.g. %cd -> cd, !python -> python.

In [ ]:
%cd ~/OpenPCDet/tools
!python demo.py --cfg_file cfgs/kitti_models/pointpillar.yaml --ckpt ../pretrained_ckpt/pointpillar_7728.pth --data_path ../data/kitti/training/velodyne/000039.bin

/userhome/35/tqwang/OpenPCDet/tools
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/.tqwang/runtime-tqwang'
2022-01-04 16:57:53,243   INFO  -----------------Quick Demo of OpenPCDet-------------------------
2022-01-04 16:57:53,245   INFO  Total number of samples: 	1
2022-01-04 16:57:55,936   INFO  ==> Loading parameters from checkpoint ../pretrained_ckpt/pointpillar_7728.pth to CPU
2022-01-04 16:57:55,963   INFO  ==> Done (loaded 127/127)
2022-01-04 16:57:56,003   INFO  Visualized sample index: 	1
/userhome/35/tqwang/OpenPCDet/pcdet/models/model_utils/model_nms_utils.py:23: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  original_idxs = scores_mask.nonzero().view(-1)
2022-01-04 16:58:08,663   INFO  Demo done.


#### You will see a mayavi window popped up to show the visualization result. 
#### Important: You have to launch this jupyter notebook file (.ipynb) from MobaXterm for Windows or XQuartz for Mac or regular terminal for Ubuntu.
<center>
    <img src="https://i.imgur.com/b5o5ssq.png" width = "80%">
    <br>
    <div style="color:orange;
    display: inline-block;
    ">Fig. Visualization result for 3D object detection in point cloud </div>
    
</center>

## 4. Training and testing given a model config

## Train model with given config (takes around 2 hours on a single gpu), train for 20 epochs.
#### You can add --extra_tag (e.g. --extra_tag experiment1), the checkpoint files will be stored after every epoch, and be stored in OpenPCDet/output/pointpillar/experiment1/ckpt/ folder

In [ ]:
%cd ~/OpenPCDet/tools/
!python train.py --cfg_file ./cfgs/kitti_models/pointpillar.yaml --extra_tag experiment1

/userhome/35/tqwang/OpenPCDet/tools
2022-01-04 16:58:33,961   INFO  **********************Start logging**********************
2022-01-04 16:58:33,962   INFO  CUDA_VISIBLE_DEVICES=0
2022-01-04 16:58:33,962   INFO  cfg_file         ./cfgs/kitti_models/pointpillar.yaml
2022-01-04 16:58:33,962   INFO  batch_size       4
2022-01-04 16:58:33,962   INFO  epochs           20
2022-01-04 16:58:33,962   INFO  workers          4
2022-01-04 16:58:33,962   INFO  extra_tag        experiment1
2022-01-04 16:58:33,962   INFO  ckpt             None
2022-01-04 16:58:33,962   INFO  pretrained_model None
2022-01-04 16:58:33,962   INFO  launcher         none
2022-01-04 16:58:33,962   INFO  tcp_port         18888
2022-01-04 16:58:33,962   INFO  sync_bn          False
2022-01-04 16:58:33,962   INFO  fix_random_seed  False
2022-01-04 16:58:33,962   INFO  ckpt_save_interval 1
2022-01-04 16:58:33,962   INFO  local_rank       0
2022-01-04 16:58:33,962   INFO  max_ckpt_save_num 30
2022-01-04 16:58:33,963   INFO  me

2022-01-04 16:58:34,250   INFO  Database filter by min points Car: 14357 => 13532
2022-01-04 16:58:34,251   INFO  Database filter by min points Pedestrian: 2207 => 2168
2022-01-04 16:58:34,251   INFO  Database filter by min points Cyclist: 734 => 705
2022-01-04 16:58:34,284   INFO  Database filter by difficulty Car: 13532 => 10759
2022-01-04 16:58:34,289   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2022-01-04 16:58:34,292   INFO  Database filter by difficulty Cyclist: 705 => 581
2022-01-04 16:58:34,301   INFO  Loading KITTI dataset
2022-01-04 16:58:34,476   INFO  Total samples for KITTI dataset: 3712
2022-01-04 16:58:37,228   INFO  PointPillar(
  (vfe): PillarVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone_3d): None
  (map_to_bev_module): PointPillarScatter()
  (pfe)

epochs:   0%| | 0/20 [00:05<?, ?it/s, loss=2.23, lr=0.0003, d_time=0.00(0.10), f
train:   1%|▎                     | 11/928 [00:05<05:34,  2.74it/s, total_it=10]
epochs:   0%| | 0/20 [00:05<?, ?it/s, loss=2.21, lr=0.0003, d_time=0.00(0.10), f
train:   1%|▎                     | 12/928 [00:05<05:24,  2.82it/s, total_it=11]
epochs:   0%| | 0/20 [00:06<?, ?it/s, loss=2.47, lr=0.0003, d_time=0.00(0.09), f
train:   1%|▎                     | 13/928 [00:06<05:14,  2.91it/s, total_it=12]
epochs:   0%| | 0/20 [00:06<?, ?it/s, loss=2.38, lr=0.0003, d_time=0.00(0.08), f
train:   2%|▎                     | 14/928 [00:06<05:16,  2.89it/s, total_it=13]
epochs:   0%| | 0/20 [00:06<?, ?it/s, loss=2.38, lr=0.0003, d_time=0.00(0.07), f
train:   2%|▎                     | 15/928 [00:06<05:08,  2.96it/s, total_it=14]
epochs:   0%| | 0/20 [00:07<?, ?it/s, loss=2.18, lr=0.0003, d_time=0.00(0.07), f
train:   2%|▍                     | 16/928 [00:07<05:00,  3.04it/s, total_it=15]
epochs:   0%| | 0/20 [00:07<

epochs:   0%| | 0/20 [00:21<?, ?it/s, loss=1.68, lr=0.0003, d_time=0.00(0.02), f
train:   6%|█▍                    | 60/928 [00:21<05:03,  2.86it/s, total_it=59]
epochs:   0%| | 0/20 [00:22<?, ?it/s, loss=1.87, lr=0.0003, d_time=0.00(0.02), f
train:   7%|█▍                    | 61/928 [00:22<04:55,  2.93it/s, total_it=60]
epochs:   0%| | 0/20 [00:22<?, ?it/s, loss=1.78, lr=0.0003, d_time=0.00(0.02), f
train:   7%|█▍                    | 62/928 [00:22<04:46,  3.02it/s, total_it=61]
epochs:   0%| | 0/20 [00:22<?, ?it/s, loss=1.72, lr=0.0003, d_time=0.00(0.02), f
train:   7%|█▍                    | 63/928 [00:22<04:53,  2.94it/s, total_it=62]
epochs:   0%| | 0/20 [00:23<?, ?it/s, loss=2.02, lr=0.0003, d_time=0.00(0.02), f
train:   7%|█▌                    | 64/928 [00:23<04:52,  2.96it/s, total_it=63]
epochs:   0%| | 0/20 [00:23<?, ?it/s, loss=1.65, lr=0.0003, d_time=0.00(0.02), f
train:   7%|█▌                    | 65/928 [00:23<04:45,  3.02it/s, total_it=64]
epochs:   0%| | 0/20 [00:23<

epochs:   0%| | 0/20 [00:38<?, ?it/s, loss=1.51, lr=0.000301, d_time=0.00(0.01),
train:  12%|██▎                 | 109/928 [00:38<04:53,  2.79it/s, total_it=108]
epochs:   0%| | 0/20 [00:38<?, ?it/s, loss=1.6, lr=0.000301, d_time=0.00(0.01), 
train:  12%|██▎                 | 110/928 [00:38<04:41,  2.91it/s, total_it=109]
epochs:   0%| | 0/20 [00:39<?, ?it/s, loss=1.46, lr=0.000301, d_time=0.00(0.01),
train:  12%|██▍                 | 111/928 [00:39<04:37,  2.94it/s, total_it=110]
epochs:   0%| | 0/20 [00:39<?, ?it/s, loss=1.56, lr=0.000301, d_time=0.00(0.01),
train:  12%|██▍                 | 112/928 [00:39<04:40,  2.91it/s, total_it=111]
epochs:   0%| | 0/20 [00:39<?, ?it/s, loss=1.57, lr=0.000301, d_time=0.00(0.01),
train:  12%|██▍                 | 113/928 [00:39<04:35,  2.96it/s, total_it=112]
epochs:   0%| | 0/20 [00:40<?, ?it/s, loss=1.46, lr=0.000302, d_time=0.00(0.01),
train:  12%|██▍                 | 114/928 [00:40<04:32,  2.98it/s, total_it=113]
epochs:   0%| | 0/20 [00:40<

...

epochs:  95%|▉| 19/20 [1:44:12<05:13, 313.14s/it, loss=0.44, lr=2.32e-6, d_time=
train:  79%|██████████████▏   | 734/928 [04:07<01:05,  2.97it/s, total_it=18365]
epochs:  95%|▉| 19/20 [1:44:13<05:13, 313.14s/it, loss=0.315, lr=2.3e-6, d_time=
train:  79%|██████████████▎   | 735/928 [04:07<01:06,  2.92it/s, total_it=18366]
epochs:  95%|▉| 19/20 [1:44:13<05:13, 313.14s/it, loss=0.426, lr=2.28e-6, d_time
train:  79%|██████████████▎   | 736/928 [04:08<01:04,  2.96it/s, total_it=18367]
epochs:  95%|▉| 19/20 [1:44:14<05:13, 313.14s/it, loss=0.514, lr=2.25e-6, d_time
train:  79%|██████████████▎   | 737/928 [04:08<01:04,  2.98it/s, total_it=18368]
epochs:  95%|▉| 19/20 [1:44:14<05:13, 313.14s/it, loss=0.43, lr=2.23e-6, d_time=
train:  80%|██████████████▎   | 738/928 [04:09<01:08,  2.77it/s, total_it=18369]
epochs:  95%|▉| 19/20 [1:44:14<05:13, 313.14s/it, loss=0.464, lr=2.21e-6, d_time
train:  80%|██████████████▎   | 739/928 [04:09<01:07,  2.82it/s, total_it=18370]
epochs:  95%|▉| 19/20 [1:44:

epochs:  95%|▉| 19/20 [1:44:29<05:13, 313.14s/it, loss=0.338, lr=1.32e-6, d_time
train:  84%|███████████████▏  | 783/928 [04:24<00:48,  2.97it/s, total_it=18414]
epochs:  95%|▉| 19/20 [1:44:29<05:13, 313.14s/it, loss=0.393, lr=1.3e-6, d_time=
train:  84%|███████████████▏  | 784/928 [04:24<00:47,  3.00it/s, total_it=18415]
epochs:  95%|▉| 19/20 [1:44:30<05:13, 313.14s/it, loss=0.443, lr=1.28e-6, d_time
train:  85%|███████████████▏  | 785/928 [04:24<00:48,  2.94it/s, total_it=18416]
epochs:  95%|▉| 19/20 [1:44:30<05:13, 313.14s/it, loss=0.459, lr=1.27e-6, d_time
train:  85%|███████████████▏  | 786/928 [04:25<00:48,  2.95it/s, total_it=18417]
epochs:  95%|▉| 19/20 [1:44:30<05:13, 313.14s/it, loss=0.485, lr=1.25e-6, d_time
train:  85%|███████████████▎  | 787/928 [04:25<00:48,  2.92it/s, total_it=18418]
epochs:  95%|▉| 19/20 [1:44:31<05:13, 313.14s/it, loss=0.425, lr=1.23e-6, d_time
train:  85%|███████████████▎  | 788/928 [04:25<00:47,  2.92it/s, total_it=18419]
epochs:  95%|▉| 19/20 [1:44:

epochs:  95%|▉| 19/20 [1:44:46<05:13, 313.14s/it, loss=0.365, lr=6.03e-7, d_time
train:  90%|████████████████▏ | 832/928 [04:40<00:32,  2.93it/s, total_it=18463]
epochs:  95%|▉| 19/20 [1:44:46<05:13, 313.14s/it, loss=0.524, lr=5.92e-7, d_time
train:  90%|████████████████▏ | 833/928 [04:41<00:31,  3.02it/s, total_it=18464]
epochs:  95%|▉| 19/20 [1:44:46<05:13, 313.14s/it, loss=0.457, lr=5.8e-7, d_time=
train:  90%|████████████████▏ | 834/928 [04:41<00:30,  3.06it/s, total_it=18465]
epochs:  95%|▉| 19/20 [1:44:47<05:13, 313.14s/it, loss=0.436, lr=5.69e-7, d_time
train:  90%|████████████████▏ | 835/928 [04:41<00:31,  2.98it/s, total_it=18466]
epochs:  95%|▉| 19/20 [1:44:47<05:13, 313.14s/it, loss=0.424, lr=5.57e-7, d_time
train:  90%|████████████████▏ | 836/928 [04:42<00:31,  2.95it/s, total_it=18467]
epochs:  95%|▉| 19/20 [1:44:47<05:13, 313.14s/it, loss=0.417, lr=5.46e-7, d_time
train:  90%|████████████████▏ | 837/928 [04:42<00:32,  2.83it/s, total_it=18468]
epochs:  95%|▉| 19/20 [1:44:

epochs:  95%|▉| 19/20 [1:45:02<05:13, 313.14s/it, loss=0.44, lr=1.73e-7, d_time=
train:  95%|█████████████████ | 881/928 [04:57<00:15,  3.06it/s, total_it=18512]
epochs:  95%|▉| 19/20 [1:45:03<05:13, 313.14s/it, loss=0.632, lr=1.68e-7, d_time
train:  95%|█████████████████ | 882/928 [04:57<00:15,  3.03it/s, total_it=18513]
epochs:  95%|▉| 19/20 [1:45:03<05:13, 313.14s/it, loss=0.587, lr=1.62e-7, d_time
train:  95%|█████████████████▏| 883/928 [04:58<00:15,  2.98it/s, total_it=18514]
epochs:  95%|▉| 19/20 [1:45:03<05:13, 313.14s/it, loss=0.396, lr=1.56e-7, d_time
train:  95%|█████████████████▏| 884/928 [04:58<00:15,  2.85it/s, total_it=18515]
epochs:  95%|▉| 19/20 [1:45:04<05:13, 313.14s/it, loss=0.412, lr=1.51e-7, d_time
train:  95%|█████████████████▏| 885/928 [04:58<00:14,  2.94it/s, total_it=18516]
epochs:  95%|▉| 19/20 [1:45:04<05:13, 313.14s/it, loss=0.402, lr=1.46e-7, d_time
train:  95%|█████████████████▏| 886/928 [04:59<00:14,  2.86it/s, total_it=18517]
epochs:  95%|▉| 19/20 [1:45:

epochs: 100%|█| 20/20 [1:45:19<00:00, 315.95s/it, loss=0.68, lr=3.01e-8, d_time=
2022-01-04 18:43:56,293   INFO  **********************End training cfgs/kitti_models/pointpillar(experiment1)**********************



2022-01-04 18:43:56,294   INFO  **********************Start evaluation cfgs/kitti_models/pointpillar(experiment1)**********************
2022-01-04 18:43:56,299   INFO  Loading KITTI dataset
2022-01-04 18:43:56,523   INFO  Total samples for KITTI dataset: 3769
2022-01-04 18:43:56,548   INFO  ==> Loading parameters from checkpoint /userhome/35/tqwang/OpenPCDet/output/pointpillar/experiment1/ckpt/checkpoint_epoch_20.pth to GPU
2022-01-04 18:43:56,656   INFO  ==> Checkpoint trained from version: pcdet+0.5.1+34c75bb
2022-01-04 18:43:56,662   INFO  ==> Done (loaded 127/127)
2022-01-04 18:43:56,666   INFO  *************** EPOCH 20 EVALUATION *****************
eval: 100%|████| 943/943 [02:06<00:00,  7.43it/s, recall_0.3=(0, 16463) / 17558]
2022-01-04 18:46:03,600   INFO  **********

### Test model with given config and checkpoint file.
#### Note that the stored checkpoint files are under OpenPCDet/output/cfgs/.... folder by default.

In [ ]:
%cd ~/OpenPCDet/tools/
!python test.py --cfg_file ./cfgs/kitti_models/pointpillar.yaml --ckpt ../output/pointpillar/experiment1/ckpt/checkpoint_epoch_20.pth

/userhome/35/tqwang/OpenPCDet/tools
2022-01-04 20:49:11,487   INFO  **********************Start logging**********************
2022-01-04 20:49:11,487   INFO  CUDA_VISIBLE_DEVICES=0
2022-01-04 20:49:11,487   INFO  cfg_file         ./cfgs/kitti_models/pointpillar.yaml
2022-01-04 20:49:11,487   INFO  batch_size       4
2022-01-04 20:49:11,487   INFO  workers          4
2022-01-04 20:49:11,487   INFO  extra_tag        default
2022-01-04 20:49:11,487   INFO  ckpt             ../output/pointpillar/experiment1/ckpt/checkpoint_epoch_20.pth
2022-01-04 20:49:11,487   INFO  launcher         none
2022-01-04 20:49:11,487   INFO  tcp_port         18888
2022-01-04 20:49:11,487   INFO  local_rank       0
2022-01-04 20:49:11,487   INFO  set_cfgs         None
2022-01-04 20:49:11,488   INFO  max_waiting_mins 30
2022-01-04 20:49:11,488   INFO  start_epoch      0
2022-01-04 20:49:11,488   INFO  eval_tag         default
2022-01-04 20:49:11,488   INFO  eval_all         False
2022-01-04 20:49:11,488   INFO  c

2022-01-04 20:49:11,705   INFO  Total samples for KITTI dataset: 3769
2022-01-04 20:49:14,780   INFO  ==> Loading parameters from checkpoint ../output/pointpillar/experiment1/ckpt/checkpoint_epoch_20.pth to GPU
2022-01-04 20:49:14,915   INFO  ==> Checkpoint trained from version: pcdet+0.5.1+34c75bb
2022-01-04 20:49:14,930   INFO  ==> Done (loaded 127/127)
2022-01-04 20:49:14,944   INFO  *************** EPOCH 20 EVALUATION *****************
eval:   0%|                                             | 0/943 [00:00<?, ?it/s]../pcdet/models/model_utils/model_nms_utils.py:23: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  original_idxs = scores_mask.nonzero().view(-1)
eval: 100%|████| 943/943 [01:47<00:00,  8.78it/s, recall_0.3=(0, 16463) / 17558]
2022-01-04 20:51:02,354   INF

## 5. Relationship between the config file and the model creation
### Let's have a look at PointPillar's config file (OpenPCDet/tools/cfg/kitti_models/pointpillar.yaml) and focus on the config for model creation
```python
MODEL:
    NAME: PointPillar

    VFE:
        NAME: PillarVFE
        WITH_DISTANCE: False
        USE_ABSLOTE_XYZ: True
        USE_NORM: True
        NUM_FILTERS: [64]

    MAP_TO_BEV:
        NAME: PointPillarScatter
        NUM_BEV_FEATURES: 64

    BACKBONE_2D:
        NAME: BaseBEVBackbone
        LAYER_NUMS: [3, 5, 5]
        LAYER_STRIDES: [2, 2, 2]
        NUM_FILTERS: [64, 128, 256]
        UPSAMPLE_STRIDES: [1, 2, 4]
        NUM_UPSAMPLE_FILTERS: [128, 128, 128]

    DENSE_HEAD:
        NAME: AnchorHeadSingle
        CLASS_AGNOSTIC: False

        USE_DIRECTION_CLASSIFIER: True
        DIR_OFFSET: 0.78539
        DIR_LIMIT_OFFSET: 0.0
        NUM_DIR_BINS: 2

        ANCHOR_GENERATOR_CONFIG: [
            {
                'class_name': 'Car',
                'anchor_sizes': [[3.9, 1.6, 1.56]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [-1.78],
                'align_center': False,
                'feature_map_stride': 2,
                'matched_threshold': 0.6,
                'unmatched_threshold': 0.45
            },
            {
                'class_name': 'Pedestrian',
                'anchor_sizes': [[0.8, 0.6, 1.73]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [-0.6],
                'align_center': False,
                'feature_map_stride': 2,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            },
            {
                'class_name': 'Cyclist',
                'anchor_sizes': [[1.76, 0.6, 1.73]],
                'anchor_rotations': [0, 1.57],
                'anchor_bottom_heights': [-0.6],
                'align_center': False,
                'feature_map_stride': 2,
                'matched_threshold': 0.5,
                'unmatched_threshold': 0.35
            }
        ]

        TARGET_ASSIGNER_CONFIG:
            NAME: AxisAlignedTargetAssigner
            POS_FRACTION: -1.0
            SAMPLE_SIZE: 512
            NORM_BY_NUM_EXAMPLES: False
            MATCH_HEIGHT: False
            BOX_CODER: ResidualCoder

        LOSS_CONFIG:
            LOSS_WEIGHTS: {
                'cls_weight': 1.0,
                'loc_weight': 2.0,
                'dir_weight': 0.2,
                'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
            }

    POST_PROCESSING:
        RECALL_THRESH_LIST: [0.3, 0.5, 0.7]
        SCORE_THRESH: 0.1
        OUTPUT_RAW_SCORE: False

        EVAL_METRIC: kitti

        NMS_CONFIG:
            MULTI_CLASSES_NMS: False
            NMS_TYPE: nms_gpu
            NMS_THRESH: 0.01
            NMS_PRE_MAXSIZE: 4096
            NMS_POST_MAXSIZE: 500
```

#### The model is defined by the congiuration parameters of all its components, including the network architecture itself such as the VFE (Voxel Feature Extrator) and some task-specific components such as the target_assigner and post processing for object detection.

#### Next, we will show how those model configs actually build the model.

#### Starting from OpenPCDet/tools/train.py, we can see this line 
```python
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=train_set)
```

#### Then, we jump to OpenPCDet/pcdet/models/\_\_init__.py where build_network is defined
```python
def build_network(model_cfg, num_class, dataset):
    model = build_detector(
        model_cfg=model_cfg, num_class=num_class, dataset=dataset
    )
    return model
```

#### Then, we jump to OpenPCDet/pcdet/models/detectors/\_\_init__.py where build_detector is defined
```python
from .detector3d_template import Detector3DTemplate
from .PartA2_net import PartA2Net
from .point_rcnn import PointRCNN
from .pointpillar import PointPillar
from .pv_rcnn import PVRCNN
from .second_net import SECONDNet
from .second_net_iou import SECONDNetIoU
from .caddn import CaDDN
from .voxel_rcnn import VoxelRCNN

__all__ = {
    'Detector3DTemplate': Detector3DTemplate,
    'SECONDNet': SECONDNet,
    'PartA2Net': PartA2Net,
    'PVRCNN': PVRCNN,
    'PointPillar': PointPillar,
    'PointRCNN': PointRCNN,
    'SECONDNetIoU': SECONDNetIoU,
    'CaDDN': CaDDN,
    'VoxelRCNN': VoxelRCNN,
}


def build_detector(model_cfg, num_class, dataset):
    model = __all__[model_cfg.NAME](
        model_cfg=model_cfg, num_class=num_class, dataset=dataset
    )

    return model
```

#### Use PointPillar as an example. Since we have specified the model name in the config file as follow:
```python
MODEL:
    NAME: PointPillar
```
#### The above line actually becomes:
```python
    model = PointPillar(model_cfg=model_cfg, num_class=num_class, dataset=dataset)
```

#### For the individual model components such as the VFE, MAP_TO_BEV, BACKBONE_2D, and etc., they all build in this similar way.

#### Taking the VFE as an example, in OpenPCDet/pcdet/models/detectors/detector3d_template.py:
```python
def build_vfe(self, model_info_dict):
    if self.model_cfg.get('VFE', None) is None:
        return None, model_info_dict

    vfe_module = vfe.__all__[self.model_cfg.VFE.NAME](
        model_cfg=self.model_cfg.VFE,
        num_point_features=model_info_dict['num_rawpoint_features'],
        point_cloud_range=model_info_dict['point_cloud_range'],
        voxel_size=model_info_dict['voxel_size'],
        grid_size=model_info_dict['grid_size'],
        depth_downsample_factor=model_info_dict['depth_downsample_factor']
    )
    model_info_dict['num_point_features'] = vfe_module.get_output_feature_dim()
    model_info_dict['module_list'].append(vfe_module)
    return vfe_module, model_info_dict
```
#### The line to build vfe_module actually becomes:
```python
    vfe_module = PillarVFE(
        model_cfg=self.model_cfg.VFE,
        num_point_features=model_info_dict['num_rawpoint_features'],
        point_cloud_range=model_info_dict['point_cloud_range'],
        voxel_size=model_info_dict['voxel_size'],
        grid_size=model_info_dict['grid_size'],
        depth_downsample_factor=model_info_dict['depth_downsample_factor']
    )
```

### PillarVFE is define in OpenPCDet/pcdet/models/backbones_3d/vfe/pillar_vfe.py
```python
class PillarVFE(VFETemplate):
    def __init__(self, model_cfg, num_point_features, voxel_size, point_cloud_range, **kwargs):
        super().__init__(model_cfg=model_cfg)

        self.use_norm = self.model_cfg.USE_NORM
        self.with_distance = self.model_cfg.WITH_DISTANCE
        self.use_absolute_xyz = self.model_cfg.USE_ABSLOTE_XYZ
        num_point_features += 6 if self.use_absolute_xyz else 3
        if self.with_distance:
            num_point_features += 1

        self.num_filters = self.model_cfg.NUM_FILTERS
        assert len(self.num_filters) > 0
        num_filters = [num_point_features] + list(self.num_filters)

        pfn_layers = []
        for i in range(len(num_filters) - 1):
            in_filters = num_filters[i]
            out_filters = num_filters[i + 1]
            pfn_layers.append(
                PFNLayer(in_filters, out_filters, self.use_norm, last_layer=(i >= len(num_filters) - 2))
            )
        self.pfn_layers = nn.ModuleList(pfn_layers)

        self.voxel_x = voxel_size[0]
        self.voxel_y = voxel_size[1]
        self.voxel_z = voxel_size[2]
        self.x_offset = self.voxel_x / 2 + point_cloud_range[0]
        self.y_offset = self.voxel_y / 2 + point_cloud_range[1]
        self.z_offset = self.voxel_z / 2 + point_cloud_range[2]

    def get_output_feature_dim(self):
        return self.num_filters[-1]

    def get_paddings_indicator(self, actual_num, max_num, axis=0):
        actual_num = torch.unsqueeze(actual_num, axis + 1)
        max_num_shape = [1] * len(actual_num.shape)
        max_num_shape[axis + 1] = -1
        max_num = torch.arange(max_num, dtype=torch.int, device=actual_num.device).view(max_num_shape)
        paddings_indicator = actual_num.int() > max_num
        return paddings_indicator

    def forward(self, batch_dict, **kwargs):
  
        voxel_features, voxel_num_points, coords = batch_dict['voxels'], batch_dict['voxel_num_points'], batch_dict['voxel_coords']
        points_mean = voxel_features[:, :, :3].sum(dim=1, keepdim=True) / voxel_num_points.type_as(voxel_features).view(-1, 1, 1)
        f_cluster = voxel_features[:, :, :3] - points_mean

        f_center = torch.zeros_like(voxel_features[:, :, :3])
        f_center[:, :, 0] = voxel_features[:, :, 0] - (coords[:, 3].to(voxel_features.dtype).unsqueeze(1) * self.voxel_x + self.x_offset)
        f_center[:, :, 1] = voxel_features[:, :, 1] - (coords[:, 2].to(voxel_features.dtype).unsqueeze(1) * self.voxel_y + self.y_offset)
        f_center[:, :, 2] = voxel_features[:, :, 2] - (coords[:, 1].to(voxel_features.dtype).unsqueeze(1) * self.voxel_z + self.z_offset)

        if self.use_absolute_xyz:
            features = [voxel_features, f_cluster, f_center]
        else:
            features = [voxel_features[..., 3:], f_cluster, f_center]

        if self.with_distance:
            points_dist = torch.norm(voxel_features[:, :, :3], 2, 2, keepdim=True)
            features.append(points_dist)
        features = torch.cat(features, dim=-1)

        voxel_count = features.shape[1]
        mask = self.get_paddings_indicator(voxel_num_points, voxel_count, axis=0)
        mask = torch.unsqueeze(mask, -1).type_as(voxel_features)
        features *= mask
        for pfn in self.pfn_layers:
            features = pfn(features)
        features = features.squeeze()
        batch_dict['pillar_features'] = features
        return batch_dict
```



## Project
### Modify the PointPillar's VFE module by inserting the TA (Tripple Attention) Module proposed in TANet, [paper link](https://arxiv.org/pdf/1912.05163.pdf)

### TANet: Robust 3D Object Detection from Point Clouds with Triple Attention
<center>
    <img src="https://i.imgur.com/BezIDFS.png" width = "80%">
    <br>
    <div style="color:orange;
    display: inline-block;
    ">Fig.Overall architecture of TANet</div>
</center>

#### The proposed TA (Tripple Attention) module in the VFE (Voxel Featutre Extractor) module is the one of the main contributions of this paper. 
<center>
    <img src="https://i.imgur.com/zt4lpcs.png" width = "40%">
    <br>
    <div style="color:orange;
    display: inline-block;
    ">Fig.Tripple Attention module for VFE (Voxel Feature Extractor)</div>
</center>

#### Visualization result proposed by TANet's authors compared to PointPillar. 
<center>
    <img src="https://i.imgur.com/YbzgaWX.png" width = "80%">
    <br>
    <div style="color:orange;
    display: inline-block;
    ">Fig.Visualization Result</div>
</center>


### Project Tasks:
### 1. Implement the TA (Tripple Attention) module.
### 2. Compare the results of the modified model and the original PointPillar in terms of both the mAP and the execution latency. (Both models train for 20 epochs)
### 3. Visualize and compare the prediction results of the modified model and the original PointPillar. Note: Use the samples in the [validation set](https://github.com/tianqi-wang1996/OpenPCDet/blob/master/data/kitti/ImageSets/val.txt) to compare. For example, the file contains 000001 meaning that OpenPCDet/data/kitti/training/velodyne/000001.bin is in the validation set.

#### To insert the TA module, we have already written and wrapped other necessary code for you. You can directly go to OpenPCDet/pcdet/models/backbones_3d/vfe/pillar_vfe.py, and fill the forward functions that are indicated by "# WRITE YOUR CODE BELOW!"
```python
# Point-wise attention for each voxel
class PALayer(nn.Module):
    def __init__(self, dim_pa, reduction_pa):
        super(PALayer, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim_pa, dim_pa // reduction_pa),
            nn.ReLU(inplace=True),
            nn.Linear(dim_pa // reduction_pa, dim_pa)
        )

    def forward(self, x):
        # WRITE YOUR CODE BELOW!
        
        return 

# Channel-wise attention for each voxel
class CALayer(nn.Module):
    def __init__(self, dim_ca, reduction_ca):
        super(CALayer, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim_ca, dim_ca // reduction_ca),
            nn.ReLU(inplace=True),
            nn.Linear(dim_ca // reduction_ca, dim_ca)
        )

    def forward(self, x):
        # WRITE YOUR CODE BELOW!

        return y


# Point-wise and Channel-wise attention for each voxel
class PACALayer(nn.Module):
    def __init__(self, dim_ca, dim_pa, reduction_r):
        super(PACALayer, self).__init__()
        self.pa = PALayer(dim_pa, dim_pa // reduction_r)
        self.ca = CALayer(dim_ca, dim_ca // reduction_r)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        # WRITE YOUR CODE BELOW!
        
        return out, paca_normal_weight


# Voxel-wise attention for each voxel
class VALayer(nn.Module):
    def __init__(self, c_num, p_num):
        super(VALayer, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(c_num + 3, 1),
            nn.ReLU(inplace=True)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(p_num, 1),
            nn.ReLU(inplace=True)
        )

        self.sigmod = nn.Sigmoid()

    def forward(self, voxel_center, paca_feat):
        '''
        :param voxel_center: size (K,1,3)
        :param SACA_Feat: size (K,N,C)
        :return: voxel_attention_weight: size (K,1,1)
        '''
        # WRITE YOUR CODE BELOW!

        return voxel_attention_weight
```

#### After finishing your code above, train & test the modified network by using the following commands
```python
cd ~/OpenPCDet/tools/
python train.py --cfg_file ./cfgs/kitti_models/pointpillar_tanet.yaml --extra_tag experiment1
```
```python
cd ~/OpenPCDet/tools/
python test.py --cfg_file ./cfgs/kitti_models/pointpillar_tanet.yaml --ckpt ../output/pointpillar_tanet/experiment1/ckpt/checkpoint_epoch_20.pth
```
```python
cd ~/OpenPCDet/tools/
python demo.py --cfg_file ./cfgs/kitti_models/pointpillar_tanet.yaml --ckpt ../output/pointpillar_tanet/experiment1/ckpt/checkpoint_epoch_20.pth --data_path ../data/kitti/training/velodyne/000039.bin
```